<center>
    Data Bowl: Best Method For Defenders
    By: Fabrice Mulumba and Andrew Lins
    App Link: <a>https://moneyteam.netlify.app/ </a>
</center>

In [ ]:
import numpy as np
import pandas as pd

# Filter 
# https://www.listendata.com/2019/07/how-to-filter-pandas-dataframe.html

In [ ]:
# games=pd.read_csv('games.csv')
# players=pd.read_csv('players.csv')
# plays=pd.read_csv('plays.csv')
# week1=pd.read_csv('week1.csv')
# week2=pd.read_csv('week2.csv')
# week3=pd.read_csv('week3.csv')
# week4=pd.read_csv('week4.csv')
# week5=pd.read_csv('week5.csv')
# week6=pd.read_csv('week6.csv')
# week7=pd.read_csv('week7.csv')
# week8=pd.read_csv('week8.csv')
# week9=pd.read_csv('week9.csv')
# week10=pd.read_csv('week10.csv')
# week11=pd.read_csv('week11.csv')
# week12=pd.read_csv('week12.csv')
# week13=pd.read_csv('week13.csv')
# week14=pd.read_csv('week14.csv')
# week15=pd.read_csv('week15.csv')
# week16=pd.read_csv('week16.csv')
# week17=pd.read_csv('week17.csv')
# games = [week1, week2, week3, week4, week5, week6, week7, week8, week9, week10, week11, week12, week13,
#         week14, week15, week16, week17]
# all_games = pd.concat(games).reset_index(drop=True)
 



In [ ]:
games=pd.read_csv('games.csv')
players=pd.read_csv('players.csv')
all_games = pd.read_csv('all_games.csv')
all_games = all_games.drop(all_games.columns[0], axis=1)
all_games

In [ ]:
plays=pd.read_csv('plays.csv')

In [ ]:
allPlaysGames = pd.merge(plays, all_games, how='inner', on=['gameId', 'playId'])
secondaryAllPlaysGames = allPlaysGames.loc[allPlaysGames.loc[:,"position"].isin(["FS","CB","S","SS","DB"]), :]
secondaryAllPlaysGamesNoDupes = secondaryAllPlaysGames.drop_duplicates(subset = ["gameId","playId","displayName","position","nflId"])

In [ ]:
goodDefPlays = plays[plays.loc[:,"offensePlayResult"] <= 3]
goodDefPlaysGames = pd.merge(goodDefPlays, all_games, how='inner', on=['gameId', 'playId'])
secondaryGoodDefPlaysGames = goodDefPlaysGames.loc[goodDefPlaysGames.loc[:,"position"].isin(["FS","CB","S","SS","DB"]), :]
secondaryGoodDefPlaysGamesNoDupes = secondaryGoodDefPlaysGames.drop_duplicates(subset = ["gameId","playId","displayName","position","nflId"])
numSecGoodPlays = secondaryGoodDefPlaysGamesNoDupes.loc[:,"nflId"].value_counts()
numSecAllPlays = secondaryAllPlaysGamesNoDupes.loc[:,"nflId"].value_counts()
secGoodPlaysTable = pd.concat([numSecGoodPlays, numSecAllPlays], axis=1, sort=False)
secGoodPlaysTable.columns =["goodPlays","allPlays"]
secGoodPlaysTable = pd.concat([secGoodPlaysTable, secGoodPlaysTable.loc[:,"goodPlays"]/secGoodPlaysTable.loc[:,"allPlays"]], axis=1, sort=False)
secGoodPlaysTable.columns =["goodPlays","allPlays","%goodPlays"]
secGoodPlaysTable = secGoodPlaysTable.rename_axis('nflId')
allPlayers = players.loc[:,["nflId","displayName","position"]]
secPlayers = allPlayers.loc[allPlayers.loc[:,"position"].isin(["FS","CB","S","SS","DB"]), :]
allPlayers.set_index('nflId', inplace=True)
secGoodPlaysTable = secGoodPlaysTable.merge(allPlayers, how='inner', on='nflId')
secGoodPlaysTable = secGoodPlaysTable.reset_index(inplace = False)
secGoodPlaysTable = secGoodPlaysTable.loc[:,["nflId","displayName","position","allPlays","goodPlays","%goodPlays"]]

In [ ]:
badDefPlays = plays.loc[plays.loc[:,"playResult"] >= plays.loc[:,"yardsToGo"], :]
badDefPlaysGames = pd.merge(badDefPlays, all_games, how='inner', on=['gameId', 'playId'])
secondaryBadDefPlaysGames = badDefPlaysGames.loc[badDefPlaysGames.loc[:,"position"].isin(["FS","CB","S","SS","DB"]), :]
secondaryBadDefPlaysGamesNoDupes = secondaryBadDefPlaysGames.drop_duplicates(subset = ["gameId","playId","displayName","position","nflId"])
numSecBadPlays = secondaryBadDefPlaysGamesNoDupes.loc[:,"nflId"].value_counts()
secBadPlaysTable = pd.concat([numSecBadPlays, numSecAllPlays], axis=1, sort=False)
secBadPlaysTable.columns =["firstDowns","allPlays"]
secBadPlaysTable = pd.concat([secBadPlaysTable, secBadPlaysTable.loc[:,"firstDowns"]/secBadPlaysTable.loc[:,"allPlays"]], axis=1, sort=False)
secBadPlaysTable.columns =["firstDowns","allPlays","%firstDowns"]
secBadPlaysTable = secBadPlaysTable.rename_axis('nflId')
secBadPlaysTable = secBadPlaysTable.merge(allPlayers, how='inner', on='nflId')
secBadPlaysTable = secBadPlaysTable.reset_index(inplace = False)
secBadPlaysTable = secBadPlaysTable.loc[:,["nflId","displayName","position","allPlays","firstDowns","%firstDowns"]]

In [ ]:
secGoodBadPlaysTable = secGoodPlaysTable.merge(secBadPlaysTable, how='inner', on=['nflId','displayName','position','allPlays'])


In [ ]:
allPlaysGamesNoDupes = allPlaysGames.drop_duplicates(subset = ["gameId","playId","nflId"])
secPlays = allPlaysGamesNoDupes.merge(secGoodBadPlaysTable, how = "inner", on=["nflId"])

In [ ]:
secPlays.groupby(['personnelD','nflId','displayName_y'])['playId'].count()
secPlaysStats = secPlays.groupby('nflId')['playResult'].sum()
secPlaysStats = secPlaysStats.reset_index(inplace = False)
secPlaysStats.columns = ["nflId","yards"]
secGoodBadPlaysTable = secGoodBadPlaysTable.merge(secPlaysStats, how='inner', on=['nflId'])
secGoodBadPlaysTable = pd.concat([secGoodBadPlaysTable,secGoodBadPlaysTable.loc[:,"yards"]/secGoodBadPlaysTable.loc[:,"allPlays"]],axis=1)


In [ ]:
secGoodBadPlaysTable.columns = ["nflId","displayName","position","allPlays",
                                        "goodPlays","%goodPlays","firstDowns",
                                "%firstDowns","yards","yardsPerPlay"]

In [ ]:
#secGoodBadPlaysTable.tail(100)
#secGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"position"] == "DB", :]

In [ ]:
secGoodBadPlaysTable.describe()
eligSecGoodBadPlaysTable = secGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"allPlays"] >= 450, :]
# number of all plays over 450, in the 70th percentilish?
eligCornersGoodBadPlaysTable = eligSecGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"position"] == "CB", :]
eligSafetiesGoodBadPlaysTable = eligSecGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"position"] != "CB", :]
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.sort_values(by='%goodPlays', ascending=False)
eligCornersGoodBadPlaysTable.reset_index(inplace = True)
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.drop('index',1)
eligCornersGoodBadPlaysTable.reset_index(inplace = True)
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.rename(columns = {"index":"goodPlaysRk"})
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.sort_values(by='%firstDowns', ascending=True)
eligCornersGoodBadPlaysTable.reset_index(inplace = True)
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.drop('index',1)
eligCornersGoodBadPlaysTable.reset_index(inplace = True)
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.rename(columns = {"index":"firstDownsRk"})
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.sort_values(by='yardsPerPlay', ascending=True)
eligCornersGoodBadPlaysTable.reset_index(inplace = True)
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.drop('index',1)
eligCornersGoodBadPlaysTable.reset_index(inplace = True)
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.rename(columns = {"index":"yardsRk"})
eligCornersGoodBadPlaysTable
eligCornersGoodBadPlaysTable = eligCornersGoodBadPlaysTable.loc[:,["nflId","displayName","position","allPlays",
                                        "goodPlays","%goodPlays","goodPlaysRk",
                                        "firstDowns","%firstDowns","firstDownsRk","yards","yardsPerPlay","yardsRk",]]
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.sort_values(by='%goodPlays', ascending=False)
eligSafetiesGoodBadPlaysTable.reset_index(inplace = True)
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.drop('index',1)
eligSafetiesGoodBadPlaysTable.reset_index(inplace = True)
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.rename(columns = {"index":"goodPlaysRk"})
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.sort_values(by='%firstDowns', ascending=True)
eligSafetiesGoodBadPlaysTable.reset_index(inplace = True)
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.drop('index',1)
eligSafetiesGoodBadPlaysTable.reset_index(inplace = True)
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.rename(columns = {"index":"firstDownsRk"})
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.sort_values(by='yardsPerPlay', ascending=True)
eligSafetiesGoodBadPlaysTable.reset_index(inplace = True)
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.drop('index',1)
eligSafetiesGoodBadPlaysTable.reset_index(inplace = True)
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.rename(columns = {"index":"yardsRk"})
eligSafetiesGoodBadPlaysTable = eligSafetiesGoodBadPlaysTable.loc[:,["nflId","displayName","position","allPlays",
                                        "goodPlays","%goodPlays","goodPlaysRk",
                                        "firstDowns","%firstDowns","firstDownsRk","yards","yardsPerPlay","yardsRk",]]
eligSecGoodBadPlaysTable = pd.concat([eligSafetiesGoodBadPlaysTable,eligCornersGoodBadPlaysTable])
secGoodBadPlaysTable = secGoodBadPlaysTable.merge(eligSecGoodBadPlaysTable, how='left', on=['nflId','displayName','position','allPlays'])
secGoodBadPlaysTable = secGoodBadPlaysTable.loc[:,["nflId","displayName","position","allPlays",
                                        "goodPlays_x","%goodPlays_x","goodPlaysRk",
                                        "firstDowns_x","%firstDowns_x","firstDownsRk",
                                        "yards_x","yardsPerPlay_x","yardsRk"]]
secGoodBadPlaysTable = secGoodBadPlaysTable.rename(columns = {"goodPlays_x":"goodPlays"})
secGoodBadPlaysTable = secGoodBadPlaysTable.rename(columns = {"%goodPlays_x":"%goodPlays"})
secGoodBadPlaysTable = secGoodBadPlaysTable.rename(columns = {"firstDowns_x":"firstDowns"})
secGoodBadPlaysTable = secGoodBadPlaysTable.rename(columns = {"%firstDowns_x":"%firstDowns"})
secGoodBadPlaysTable = secGoodBadPlaysTable.rename(columns = {"yards_x":"yards"})
secGoodBadPlaysTable = secGoodBadPlaysTable.rename(columns = {"yardsPerPlay_x":"yardsPerPlay"})

In [ ]:
top15GoodPlaySec = eligSecGoodBadPlaysTable.loc[eligSecGoodBadPlaysTable.loc[:,"goodPlaysRk"] < 15, "nflId":"goodPlaysRk"]
top15GoodPlaySafeties = eligSafetiesGoodBadPlaysTable.loc[eligSafetiesGoodBadPlaysTable.loc[:,"goodPlaysRk"] < 15, "nflId":"goodPlaysRk"]
top15GoodPlayCorners =  eligCornersGoodBadPlaysTable.loc[eligCornersGoodBadPlaysTable.loc[:,"goodPlaysRk"] < 15, "nflId":"goodPlaysRk"]

In [ ]:
top25Players = top15GoodPlaySec.sort_values(by='%goodPlays', ascending=False).reset_index(inplace = False)

In [ ]:
top25Players = top25Players.drop('index',1)
top25Players = top25Players.head(25)

In [ ]:
top25Players

In [ ]:
top25PlayersPlays = secPlays.merge(top25Players, how = "inner", on=["nflId"])

In [ ]:
top25PlayerPersonnel = pd.DataFrame(top25PlayersPlays.groupby(['nflId','displayName_y','personnelD'])['playId'].count().sort_values(ascending=False))

In [ ]:
top25PlayerPersonnel = top25PlayerPersonnel.sort_values(by=["nflId","playId"], ascending=False)

In [ ]:
top25PlayerPersonnel

In [ ]:
rank = pd.DataFrame(top25PlayerPersonnel.groupby(["nflId","displayName_y"])['playId'].rank(method='first'))

In [ ]:
rank.loc[:,"playId"]

In [ ]:
secEligGoodBadPlaysTableEQ = secGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"allPlays"] >= 450, :].sort_values(by='yardsPerPlay', ascending=True)
secEligGoodBadPlaysTableEQ = pd.concat([secEligGoodBadPlaysTableEQ,(secEligGoodBadPlaysTableEQ.loc[:,"yardsRk"]+1)*(1/3) + (secEligGoodBadPlaysTableEQ.loc[:,"goodPlaysRk"]+1)*(1/3) + (secEligGoodBadPlaysTableEQ.loc[:,"firstDownsRk"]+1)*(1/3)],axis=1)
secEligGoodBadPlaysTableEQ = secEligGoodBadPlaysTableEQ.set_axis([*secEligGoodBadPlaysTableEQ.columns[:-1], 'combRk'], axis=1, inplace=False)
secEligGoodBadPlaysTableEQ=secEligGoodBadPlaysTableEQ[['nflId', 'displayName', 'position','combRk','allPlays','goodPlays','%goodPlays','goodPlaysRk','firstDowns','%firstDowns','firstDownsRk','yards','yardsPerPlay','yardsRk']]
secEligGoodBadPlaysTableEQ = secEligGoodBadPlaysTableEQ.sort_values(by='combRk', ascending=True)
secEligGoodBadPlaysTableEQ.reset_index(inplace = True)
secEligGoodBadPlaysTableEQ = secEligGoodBadPlaysTableEQ.drop('index',1)
# secEligGoodBadPlaysTableEQ.head(25)

In [ ]:
cornerEligGoodBadPlaysTableEQ = secEligGoodBadPlaysTableEQ.loc[secEligGoodBadPlaysTableEQ.loc[:,"position"] == "CB", :]
safetyEligGoodBadPlaysTableEQ = secEligGoodBadPlaysTableEQ.loc[secEligGoodBadPlaysTableEQ.loc[:,"position"] != "CB", :]
topCornerEligGoodBadPlaysTableEQ = cornerEligGoodBadPlaysTableEQ.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
topsafetyEligGoodBadPlaysTableEQ = safetyEligGoodBadPlaysTableEQ.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
#topCornerEligGoodBadPlaysTableEQ.to_json(path_or_buf='cornersEQ.json', orient='records')
#topsafetyEligGoodBadPlaysTableEQ.to_json(path_or_buf='safetiesEQ.json',orient='records')

In [ ]:
secEligGoodBadPlaysTableYGF = secGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"allPlays"] >= 450, :].sort_values(by='yardsPerPlay', ascending=True)
secEligGoodBadPlaysTableYGF = pd.concat([secEligGoodBadPlaysTableYGF,(secEligGoodBadPlaysTableYGF.loc[:,"yardsRk"]+1)*(3/6) + (secEligGoodBadPlaysTableYGF.loc[:,"goodPlaysRk"]+1)*(2/6) + (secEligGoodBadPlaysTableYGF.loc[:,"firstDownsRk"]+1)*(1/6)],axis=1)
secEligGoodBadPlaysTableYGF = secEligGoodBadPlaysTableYGF.set_axis([*secEligGoodBadPlaysTableYGF.columns[:-1], 'combRk'], axis=1, inplace=False)
secEligGoodBadPlaysTableYGF=secEligGoodBadPlaysTableYGF[['nflId', 'displayName', 'position','combRk','allPlays','goodPlays','%goodPlays','goodPlaysRk','firstDowns','%firstDowns','firstDownsRk','yards','yardsPerPlay','yardsRk']]
secEligGoodBadPlaysTableYGF = secEligGoodBadPlaysTableYGF.sort_values(by='combRk', ascending=True)
secEligGoodBadPlaysTableYGF.reset_index(inplace = True)
secEligGoodBadPlaysTableYGF = secEligGoodBadPlaysTableYGF.drop('index',1)
secEligGoodBadPlaysTableYGF.head(25)

In [ ]:
cornerEligGoodBadPlaysTableYGF = secEligGoodBadPlaysTableYGF.loc[secEligGoodBadPlaysTableYGF.loc[:,"position"] == "CB", :]
safetyEligGoodBadPlaysTableYGF = secEligGoodBadPlaysTableYGF.loc[secEligGoodBadPlaysTableYGF.loc[:,"position"] != "CB", :]
topCornerEligGoodBadPlaysTableYGF = cornerEligGoodBadPlaysTableYGF.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
topsafetyEligGoodBadPlaysTableYGF = safetyEligGoodBadPlaysTableYGF.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
#topCornerEligGoodBadPlaysTableYGF.to_json(path_or_buf='cornersYGF.json', orient='records')
#topsafetyEligGoodBadPlaysTableYGF.to_json(path_or_buf='safetiesYGF.json',orient='records')

In [ ]:
secEligGoodBadPlaysTableYFG = secGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"allPlays"] >= 450, :].sort_values(by='yardsPerPlay', ascending=True)
secEligGoodBadPlaysTableYFG = pd.concat([secEligGoodBadPlaysTableYFG,(secEligGoodBadPlaysTableYFG.loc[:,"yardsRk"]+1)*(3/6) + (secEligGoodBadPlaysTableYFG.loc[:,"goodPlaysRk"]+1)*(1/6) + (secEligGoodBadPlaysTableYFG.loc[:,"firstDownsRk"]+1)*(2/6)],axis=1)
secEligGoodBadPlaysTableYFG = secEligGoodBadPlaysTableYFG.set_axis([*secEligGoodBadPlaysTableYFG.columns[:-1], 'combRk'], axis=1, inplace=False)
secEligGoodBadPlaysTableYFG=secEligGoodBadPlaysTableYFG[['nflId', 'displayName', 'position','combRk','allPlays','goodPlays','%goodPlays','goodPlaysRk','firstDowns','%firstDowns','firstDownsRk','yards','yardsPerPlay','yardsRk']]
secEligGoodBadPlaysTableYFG = secEligGoodBadPlaysTableYFG.sort_values(by='combRk', ascending=True)
secEligGoodBadPlaysTableYFG.reset_index(inplace = True)
secEligGoodBadPlaysTableYFG = secEligGoodBadPlaysTableYFG.drop('index',1)
secEligGoodBadPlaysTableYFG.head(25)

In [ ]:
cornerEligGoodBadPlaysTableYFG = secEligGoodBadPlaysTableYFG.loc[secEligGoodBadPlaysTableYFG.loc[:,"position"] == "CB", :]
safetyEligGoodBadPlaysTableYFG = secEligGoodBadPlaysTableYFG.loc[secEligGoodBadPlaysTableYFG.loc[:,"position"] != "CB", :]
topCornerEligGoodBadPlaysTableYFG = cornerEligGoodBadPlaysTableYFG.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
topsafetyEligGoodBadPlaysTableYFG = safetyEligGoodBadPlaysTableYFG.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
#topCornerEligGoodBadPlaysTableYFG.to_json(path_or_buf='cornersYFG.json', orient='records')
#topsafetyEligGoodBadPlaysTableYFG.to_json(path_or_buf='safetiesYFG.json',orient='records')

In [ ]:
secEligGoodBadPlaysTableGYF = secGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"allPlays"] >= 450, :].sort_values(by='yardsPerPlay', ascending=True)
secEligGoodBadPlaysTableGYF = pd.concat([secEligGoodBadPlaysTableGYF,(secEligGoodBadPlaysTableGYF.loc[:,"yardsRk"]+1)*(2/6) + (secEligGoodBadPlaysTableGYF.loc[:,"goodPlaysRk"]+1)*(3/6) + (secEligGoodBadPlaysTableGYF.loc[:,"firstDownsRk"]+1)*(1/6)],axis=1)
secEligGoodBadPlaysTableGYF = secEligGoodBadPlaysTableGYF.set_axis([*secEligGoodBadPlaysTableGYF.columns[:-1], 'combRk'], axis=1, inplace=False)
secEligGoodBadPlaysTableGYF=secEligGoodBadPlaysTableGYF[['nflId', 'displayName', 'position','combRk','allPlays','goodPlays','%goodPlays','goodPlaysRk','firstDowns','%firstDowns','firstDownsRk','yards','yardsPerPlay','yardsRk']]
secEligGoodBadPlaysTableGYF = secEligGoodBadPlaysTableGYF.sort_values(by='combRk', ascending=True)
secEligGoodBadPlaysTableGYF.reset_index(inplace = True)
secEligGoodBadPlaysTableGYF = secEligGoodBadPlaysTableGYF.drop('index',1)
secEligGoodBadPlaysTableGYF.head(25)

In [ ]:
cornerEligGoodBadPlaysTableGYF = secEligGoodBadPlaysTableGYF.loc[secEligGoodBadPlaysTableGYF.loc[:,"position"] == "CB", :]
safetyEligGoodBadPlaysTableGYF = secEligGoodBadPlaysTableGYF.loc[secEligGoodBadPlaysTableGYF.loc[:,"position"] != "CB", :]
topCornerEligGoodBadPlaysTableGYF = cornerEligGoodBadPlaysTableGYF.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
topsafetyEligGoodBadPlaysTableGYF = safetyEligGoodBadPlaysTableGYF.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
#topCornerEligGoodBadPlaysTableGYF.to_json(path_or_buf='cornersGYF.json', orient='records')
#topsafetyEligGoodBadPlaysTableGYF.to_json(path_or_buf='safetiesGYF.json',orient='records')

In [ ]:
secEligGoodBadPlaysTableGFY = secGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"allPlays"] >= 450, :].sort_values(by='yardsPerPlay', ascending=True)
secEligGoodBadPlaysTableGFY = pd.concat([secEligGoodBadPlaysTableGFY,(secEligGoodBadPlaysTableGFY.loc[:,"yardsRk"]+1)*(1/6) + (secEligGoodBadPlaysTableGFY.loc[:,"goodPlaysRk"]+1)*(3/6) + (secEligGoodBadPlaysTableGFY.loc[:,"firstDownsRk"]+1)*(2/6)],axis=1)
secEligGoodBadPlaysTableGFY = secEligGoodBadPlaysTableGFY.set_axis([*secEligGoodBadPlaysTableGFY.columns[:-1], 'combRk'], axis=1, inplace=False)
secEligGoodBadPlaysTableGFY=secEligGoodBadPlaysTableGFY[['nflId', 'displayName', 'position','combRk','allPlays','goodPlays','%goodPlays','goodPlaysRk','firstDowns','%firstDowns','firstDownsRk','yards','yardsPerPlay','yardsRk']]
secEligGoodBadPlaysTableGFY = secEligGoodBadPlaysTableGFY.sort_values(by='combRk', ascending=True)
secEligGoodBadPlaysTableGFY.reset_index(inplace = True)
secEligGoodBadPlaysTableGFY = secEligGoodBadPlaysTableGFY.drop('index',1)
secEligGoodBadPlaysTableGFY.head(25)

In [ ]:
cornerEligGoodBadPlaysTableGFY = secEligGoodBadPlaysTableGFY.loc[secEligGoodBadPlaysTableGFY.loc[:,"position"] == "CB", :]
safetyEligGoodBadPlaysTableGFY = secEligGoodBadPlaysTableGFY.loc[secEligGoodBadPlaysTableGFY.loc[:,"position"] != "CB", :]
topCornerEligGoodBadPlaysTableGFY = cornerEligGoodBadPlaysTableGFY.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
topsafetyEligGoodBadPlaysTableGFY = safetyEligGoodBadPlaysTableGFY.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
#topCornerEligGoodBadPlaysTableGFY.to_json(path_or_buf='cornersGFY.json', orient='records')
#topsafetyEligGoodBadPlaysTableGFY.to_json(path_or_buf='safetiesGFY.json',orient='records')

In [ ]:
secEligGoodBadPlaysTableFGY = secGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"allPlays"] >= 450, :].sort_values(by='yardsPerPlay', ascending=True)
secEligGoodBadPlaysTableFGY = pd.concat([secEligGoodBadPlaysTableFGY,(secEligGoodBadPlaysTableFGY.loc[:,"yardsRk"]+1)*(1/6) + (secEligGoodBadPlaysTableFGY.loc[:,"goodPlaysRk"]+1)*(2/6) + (secEligGoodBadPlaysTableFGY.loc[:,"firstDownsRk"]+1)*(3/6)],axis=1)
secEligGoodBadPlaysTableFGY = secEligGoodBadPlaysTableFGY.set_axis([*secEligGoodBadPlaysTableFGY.columns[:-1], 'combRk'], axis=1, inplace=False)
secEligGoodBadPlaysTableFGY=secEligGoodBadPlaysTableFGY[['nflId', 'displayName', 'position','combRk','allPlays','goodPlays','%goodPlays','goodPlaysRk','firstDowns','%firstDowns','firstDownsRk','yards','yardsPerPlay','yardsRk']]
secEligGoodBadPlaysTableFGY = secEligGoodBadPlaysTableFGY.sort_values(by='combRk', ascending=True)
secEligGoodBadPlaysTableFGY.reset_index(inplace = True)
secEligGoodBadPlaysTableFGY = secEligGoodBadPlaysTableFGY.drop('index',1)
secEligGoodBadPlaysTableFGY.head(25)

In [ ]:
cornerEligGoodBadPlaysTableFGY = secEligGoodBadPlaysTableFGY.loc[secEligGoodBadPlaysTableFGY.loc[:,"position"] == "CB", :]
safetyEligGoodBadPlaysTableFGY = secEligGoodBadPlaysTableFGY.loc[secEligGoodBadPlaysTableFGY.loc[:,"position"] != "CB", :]
topCornerEligGoodBadPlaysTableFGY = cornerEligGoodBadPlaysTableFGY.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
topsafetyEligGoodBadPlaysTableFGY = safetyEligGoodBadPlaysTableFGY.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
#topCornerEligGoodBadPlaysTableFGY.to_json(path_or_buf='cornersFGY.json', orient='records')
#topsafetyEligGoodBadPlaysTableFGY.to_json(path_or_buf='safetiesFGY.json',orient='records')

In [ ]:
secEligGoodBadPlaysTableFYG = secGoodBadPlaysTable.loc[secGoodBadPlaysTable.loc[:,"allPlays"] >= 450, :].sort_values(by='yardsPerPlay', ascending=True)
secEligGoodBadPlaysTableFYG = pd.concat([secEligGoodBadPlaysTableFYG,(secEligGoodBadPlaysTableFYG.loc[:,"yardsRk"]+1)*(2/6) + (secEligGoodBadPlaysTableFYG.loc[:,"goodPlaysRk"]+1)*(1/6) + (secEligGoodBadPlaysTableFYG.loc[:,"firstDownsRk"]+1)*(3/6)],axis=1)
secEligGoodBadPlaysTableFYG = secEligGoodBadPlaysTableFYG.set_axis([*secEligGoodBadPlaysTableFYG.columns[:-1], 'combRk'], axis=1, inplace=False)
secEligGoodBadPlaysTableFYG=secEligGoodBadPlaysTableFYG[['nflId', 'displayName', 'position','combRk','allPlays','goodPlays','%goodPlays','goodPlaysRk','firstDowns','%firstDowns','firstDownsRk','yards','yardsPerPlay','yardsRk']]
secEligGoodBadPlaysTableFYG = secEligGoodBadPlaysTableFYG.sort_values(by='combRk', ascending=True)
secEligGoodBadPlaysTableFYG.reset_index(inplace = True)
secEligGoodBadPlaysTableFYG = secEligGoodBadPlaysTableFYG.drop('index',1)
secEligGoodBadPlaysTableFYG.head(25)

<center>    
    <h1>Top Secondary Defenders in the NFL</h1>
    <b>
        As we all know, "Defense wins championships!" But what kind of defensive stats are important? Do interceptions really make one defender better than the next? What if a certain defender is in a coference of weak quaterbacks that are extremely likely to throw interceptions based on historical stats?
    
    From playing and watching thousands of hours of football, we know what really makes a defender better than the next in comparison to determining the former with your typical stats of such as tackles, interceptions, and deflections. What really matters is what players get the defense off the field as soon as possible (3 and Outs), who limits offenses to the fewest first downs, and who allows the fewest yards. But which of these 3 categories is the most important? Are all 3 equally important?

    Today, we try to answer the question above. After reviewing all plays for secondary defenders, we determined eligible defenders must meet the following criteria: 1) CB/FS/SS/S, 2) Players must have been on the field for 450 plays or more (70th percentile). In regards to our research, we use the following terms: 1) Good Play (a play that results in an offensive advancement of 3 or less yards) and 2) First Down (a play where the offense advances greater than or equal to the yards to gain). If a defender can allow 3 or less yards 3 plays in a row, the defense will likely get off the field. Additionally, we determined total yards given up for each defender. After doing so, we determined the proportion of Good Plays to Total Plays and First Downs to Total Plays per defender. Defenders were split into two groups: Corners and Safeties, and then assigned ranks from 0.0 (First) to n-1 (Last) for Good Play%, First Down%, and Yards Allowed. Lastly, we ran the following 7 analyses with their respective weights to form Combined Ranks per secondary group:
        
        Analysis 1) 33.3% Good Play Rank, 33.3% First Down, 33.3% Yards Allowed
        Analysis 2) 50.0% Good Play Rank, 33.3% First Down, 16.7% Yards Allowed
        Analysis 3) 50.0% Good Play Rank, 16.7% First Down, 33.3% Yards Allowed
        Analysis 4) 33.3% Good Play Rank, 50.0% First Down, 16.7% Yards Allowed
        Analysis 5) 16.7% Good Play Rank, 50.0% First Down, 33.3% Yards Allowed
        Analysis 6) 33.3% Good Play Rank, 16.7% First Down, 50.0% Yards Allowed
        Analysis 7) 16.7% Good Play Rank, 33.3% First Down, 50.0% Yards Allowed
        
    For each analysis, our Top 10 Defenders remained the same. We feel extremely comfortable calling these 10 defenders the best defenders in the NFL regardless of who the leading tackler, interceptor, or deflector in the NFL
    </b>
</center>

<center>    
    <h1>Analysis 1</h1>
</center>

In [ ]:
df1 = secEligGoodBadPlaysTableEQ.head(25)
df1.to_json (r'equalsplitplays.json', orient='records')


In [ ]:
df1

<center>    
    <h1>Analysis 2</h1>
</center>

In [ ]:
secEligGoodBadPlaysTableGFY.head(25)
df2 = secEligGoodBadPlaysTableGFY.head(25)
df2.to_json (r'analysis2.json', orient='records')

In [ ]:
df2

<center>    
    <h1>Analysis 3</h1>
</center>

In [ ]:
secEligGoodBadPlaysTableGYF.head(25)
df3 = secEligGoodBadPlaysTableGYF.head(25)
df3.to_json (r'analysis3.json', orient='records')

In [ ]:
df3

<center>    
    <h1>Analysis 4</h1>
</center>

In [ ]:
secEligGoodBadPlaysTableFGY.head(25)
df4 = secEligGoodBadPlaysTableFGY.head(25)
df4.to_json (r'analysis4.json', orient='records')

In [ ]:
df4

<center>    
    <h1>Analysis 5</h1>
</center>

In [ ]:
secEligGoodBadPlaysTableFYG.head(25)
df5 = secEligGoodBadPlaysTableFYG.head(25)
df5.to_json (r'analysis5.json', orient='records')

In [ ]:
df5

<center>    
    <h1>Analysis 6</h1>
</center>

In [ ]:
secEligGoodBadPlaysTableYGF.head(25)
df6 = secEligGoodBadPlaysTableYGF.head(25)
df6.to_json (r'analysis6.json', orient='records')

In [ ]:
df6

<center>    
    <h1>Analysis 7</h1>
</center>

In [ ]:
secEligGoodBadPlaysTableYFG.head(25)
df7 = secEligGoodBadPlaysTableYFG.head(25)
df7.to_json (r'analysis7.json', orient='records')

In [ ]:
df7

In [ ]:
cornerEligGoodBadPlaysTableFYG = secEligGoodBadPlaysTableFYG.loc[secEligGoodBadPlaysTableFYG.loc[:,"position"] == "CB", :]
safetyEligGoodBadPlaysTableFYG = secEligGoodBadPlaysTableFYG.loc[secEligGoodBadPlaysTableFYG.loc[:,"position"] != "CB", :]
topCornerEligGoodBadPlaysTableFYG = cornerEligGoodBadPlaysTableFYG.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
topsafetyEligGoodBadPlaysTableFYG = safetyEligGoodBadPlaysTableFYG.merge(top25Players, how = "inner", on=['nflId', 'displayName', 'position','allPlays','goodPlays','%goodPlays','goodPlaysRk'])
#topCornerEligGoodBadPlaysTableFYG.to_json(path_or_buf='cornersFYG.json', orient='records')
#topsafetyEligGoodBadPlaysTableFYG.to_json(path_or_buf='safetiesFYG.json',orient='records')

In [ ]:
ballSnapPlays = allPlaysGames.loc[allPlaysGames.loc[:,"event"] == "ball_snap", :]

In [ ]:
ballLocationPlays = ballSnapPlays.loc[ballSnapPlays.loc[:,"displayName"] == "Football", ["playId","gameId","x"]]

In [ ]:
ballSnapPlays.describe()

In [ ]:
ballLocationPlays.columns= ["playId","gameId","xFootball"]
ballLocationPlays

In [ ]:
ballSnapPlays = ballSnapPlays.merge(ballLocationPlays, how='inner', on=['playId','gameId'])
ballSnapPlays

In [ ]:
ballSnapPlays = pd.concat([ballSnapPlays,abs(ballSnapPlays.loc[:,"x"]-ballSnapPlays.loc[:,"xFootball"])],axis=1)

In [ ]:
ballSnapPlays.head()

In [ ]:
ballSnapPlays = ballSnapPlays.set_axis([*ballSnapPlays.columns[:-1], 'yardsOffBall'], axis=1, inplace=False)


In [ ]:
secBallSnapPlays = ballSnapPlays.loc[ballSnapPlays.loc[:,"position"].isin(["FS","CB","S","SS"]), :]

In [ ]:
ballSnapPlays

In [ ]:
cornerBallSnapPlays = secBallSnapPlays.loc[secBallSnapPlays.loc[:,"position"] == 'CB', :]

In [ ]:
cornerPressBallSnapPlays = cornerBallSnapPlays.loc[cornerBallSnapPlays.loc[:,"yardsOffBall"] <= 1.5, :]

In [ ]:
cornerOffBallSnapPlays = cornerBallSnapPlays.loc[cornerBallSnapPlays.loc[:,"yardsOffBall"] > 1.5, :]

In [ ]:
cornerGoodPressBallSnapPlays = cornerPressBallSnapPlays.loc[cornerPressBallSnapPlays.loc[:,"offensePlayResult"] <= 3, :]

In [ ]:
cornerGoodOffBallSnapPlays = cornerOffBallSnapPlays.loc[cornerOffBallSnapPlays.loc[:,"offensePlayResult"] <= 3, :]

In [ ]:
cornerFirstPressBallSnapPlays = cornerPressBallSnapPlays.loc[cornerPressBallSnapPlays.loc[:,"playResult"] >= cornerPressBallSnapPlays.loc[:,"yardsToGo"], :]

In [ ]:
cornerFirstOffBallSnapPlays = cornerOffBallSnapPlays.loc[cornerOffBallSnapPlays.loc[:,"playResult"] >= cornerOffBallSnapPlays.loc[:,"yardsToGo"], :]

In [ ]:
b = cornerPressBallSnapPlays.loc[:,"displayName"].value_counts()

In [ ]:
a = cornerBallSnapPlays.loc[:,"displayName"].value_counts()

In [ ]:
d = cornerFirstPressBallSnapPlays.loc[:,"displayName"].value_counts()

In [ ]:
c = cornerGoodPressBallSnapPlays.loc[:,"displayName"].value_counts()

In [ ]:
e = cornerOffBallSnapPlays.loc[:,"displayName"].value_counts()

In [ ]:
f = cornerGoodOffBallSnapPlays.loc[:,"displayName"].value_counts()

In [ ]:
g = d = cornerFirstOffBallSnapPlays.loc[:,"displayName"].value_counts()

In [ ]:
a = pd.DataFrame(a)

In [ ]:
cornersBallSnapPlays = pd.merge(a, b, right_index = True, 
               left_index = True, how='left')

In [ ]:
cornersBallSnapPlays = pd.merge(cornersBallSnapPlays, c, right_index = True, 
               left_index = True, how='left')

In [ ]:
cornersBallSnapPlays = pd.merge(cornersBallSnapPlays, d, right_index = True, 
               left_index = True,how='left')

In [ ]:
cornersBallSnapPlays = pd.merge(cornersBallSnapPlays, e, right_index = True, 
               left_index = True,how='left')

In [ ]:
cornersBallSnapPlays = pd.merge(cornersBallSnapPlays, f, right_index = True, 
               left_index = True,how='left')

In [ ]:
cornersBallSnapPlays = pd.merge(cornersBallSnapPlays, g, right_index = True, 
               left_index = True,how='left')

In [ ]:
cornersBallSnapPlays = cornersBallSnapPlays.set_axis([*cornersBallSnapPlays.columns[:-7], ["allPlays","pressPlays","goodPressPlays","firstDownPressPlays","offPlays","goodOffPlays","firstDownOffPlays"]], axis=1, inplace=False)

In [ ]:
cornersBallSnapPlays = cornersBallSnapPlays.reset_index(inplace = False).reset_index()

In [ ]:
cornersBallSnapPlays = cornersBallSnapPlays.drop('level_0',1)

In [ ]:
cornersBallSnapPlays.columns

In [ ]:
numSecAllPlays

In [ ]:
numSecGoodPlays

In [ ]:
secGoodPlaysTable.tail(60)